#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load huggingface embedding model (``model_name="sentence-transformers/all-mpnet-base-v2"``)
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

# ADD HERE YOUR CODE
embedding_model = ...

c:\Users\sschn\OneDrive\Documents\Studium\3.Semester\Technische Informatik\Chatbot\TeamNovaBot\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\sschn\OneDrive\Documents\Studium\3.Semester\Technische Informatik\Chatbot\TeamNovaBot\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
text = "This is a test document."

# ADD HERE YOUR CODE
# Perform vector search
query_vector = ...

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: ``PyPDFLoader``
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
# ADD HERE YOUR CODE
loader = ...

# Load and split documents in chunks
# ADD HERE YOUR CODE
pages_chunked = ...

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(text):
    # Remove surrogate pairs
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

pages_chunked_cleaned = [clean_text(chunk.page_content) for chunk in pages_chunked]

print(pages_chunked[0])

In [4]:
print(pages_chunked[1])

In [5]:
print(f"Number of text chunks: {len(pages_chunked)}")

#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)

In [6]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# ADD HERE YOUR CODE
# Create a collection
collection = ...

# ADD HERE YOUR CODE
# Create chromadb
vector_db_from_client = Chroma(...)

In [7]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_chunked[:50]))]

# ADD HERE YOUR CODE
vector_db_from_client.add_documents(...)

['ce1cee73-6a7f-4fa6-81ad-30a5a4ef6fd3',
 'f212b042-aa4a-497a-9927-95c1ee22c71e',
 'f35d603c-645a-490c-ae6a-3272f69c08d7',
 '4002aaf9-bcdf-47c8-bea3-27e61c4b8853',
 'ebdf6f60-25db-4f23-95ef-f6be13f8a3dd',
 '82b4772c-6da3-4dee-a681-c928aeec9e5c',
 '2f6dcdd0-cdf6-41b6-8c9d-8353355853f6',
 'bdbf3ed4-4d6f-4bde-a0d1-2fa7e0f39fe8',
 '9938421e-6d28-4aa9-9128-3bc08b569faf',
 '04c404b4-6ea5-47d1-99a7-f4064e472eb7',
 '63eceb8b-14d8-4608-bfec-3952e4e98f53',
 '716839b7-bf31-4b92-8bd3-d4e69a1dd259',
 '0aa96591-4bb3-4696-805b-9c18deedc764',
 'ce6d1790-08bf-454b-8974-5b6d91d74a2e',
 '0242631b-6ef5-4c21-a267-3a614047d0bc',
 '07624a6b-5d4c-406e-a8c2-d51ec60121a6',
 '7f165c01-b854-4899-85fa-9a3ed6f95c57',
 '0ffd32c8-9f0e-4213-a20d-5cbe4b27f296',
 '0187cf9c-56ab-46cf-aa1b-85f0f71f8254',
 '451ad91a-57ff-4dbb-a408-443f68eccb47',
 '5c5001e1-31f5-4f70-8ca9-a485d862af78',
 'fbba6ebd-5be1-46ab-a00a-b727dbe61a7d',
 '19a685aa-8fbd-4169-8698-bd9e92748c52',
 'e502a974-1502-44f7-ac14-a7c6617d9ce8',
 'dba1a416-34a7-

In [8]:
client.count_collections()

1

In [89]:
# client.delete_collection("ai_model_book")

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents


**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)

In [9]:
search_query = "Types of Machine Learning Systems"

results = vector_db_from_client.similarity_search(...)

for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")

Types of Machine Learning Systems
There are so many different types of Machine Learning systems that it is useful to
classify them in broad categories based on:
Whether or not they are trained with human supervision (supervised, unsuper
vised, semisupervised, and Reinforcement Learning)
Whether or not they can learn incrementally on the fly (online versus batch
learning)
Whether they work by simply comparing new data points to known data points,
or instead detect patterns in the training data and build a predictive model, much
like scientists do (instance-based versus model-based learning)
These criteria are not exclusive; you can combine them in any way you like. For
example, a state-of-the-art spam filter may learn on the fly using a deep neural net
work model trained using examples of spam and ham; this makes it an online, model-
based, supervised learning system.
Lets look at each of these criteria a bit more closely.
Supervised/Unsupervised Learning
Machine Learning systems can be